In [1]:
from os import listdir
from os.path import isfile, join
import json
import re
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string 
import os
os.environ["CUDA_VISIBLE_DEVICES"]="5"
import pickle

In [ ]:
DATA = "ARTINT"

In [2]:
!nvidia-smi

Sun Jan 13 11:59:36 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.111                Driver Version: 384.111                   |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:04:00.0  On |                  N/A |
|  0%   53C    P2    62W / 250W |   4987MiB / 11172MiB |     18%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:05:00.0 Off |                  N/A |
| 28%   49C    P2    59W / 250W |   5221MiB / 11172MiB |     22%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G

In [3]:
img_text_path_1 = DATA+"train/positive_image_caption"
img_text_path_2 = DATA+"/train/negative_image_caption"

In [4]:
def preprocess(sentence):
    words = []  
    w = word_tokenize(sentence)
    w = [word.lower() for word in w]
    w = [word for word in w if word.isalpha()]
    stop_words = stopwords.words('english')
    w = [word for word in w if not word in stop_words]
    words.extend(w)
    return words

    
def bagofwords(sentence, words):
    bag = np.zeros(len(words))
    for sw in sentence:
        for i,word in enumerate(words):
            if word == sw: 
                bag[i] = 1                
    return np.array(bag)

In [5]:
def get_unigrams(x,path,image_caption,prefix):

    for file in x:
        key = prefix+file[:len(file)-5]+".pdf.json"
        image_caption[key] = ""
        with open(join(path,file),encoding="utf8") as input_file:
            data = json.load(input_file)
            for x in range(len(data)):
                if data[x]['caption'] is not None:
                    image_caption[key] = image_caption[key] + "".join(data[x]['caption'].split('.')[2:])
               
            image_caption[key] = preprocess(image_caption[key])
           

In [6]:
#for  jnca get unigrams or each element is dictionary of prefix_filename and corresponding unigram words preprocessed
x = listdir(img_text_path_1)
prefix = 'pos_'
image_caption = {}
get_unigrams(x,img_text_path_1,image_caption,prefix)
#for comnet get unigrams or each element is dictionary of prefix_filename and corresponding unigram words preprocessed
x = listdir(img_text_path_2)
prefix = 'neg_'
get_unigrams(x,img_text_path_2,image_caption,prefix)


In [7]:
def generate_vocabulary(vocabulary_name,document_unigram,prefix,x):
    for file in x:
        vocabulary_name = vocabulary_name + document_unigram[prefix+file[:len(file)-5]+".pdf.json"]
    return vocabulary_name


In [8]:
img_caption_vocabulary = []

x = listdir(img_text_path_1)
img_caption_vocabulary = generate_vocabulary(img_caption_vocabulary,image_caption,'pos_',x)

x = listdir(img_text_path_2)
img_caption_vocabulary = generate_vocabulary(img_caption_vocabulary,image_caption,'neg_',x)


In [9]:

img_caption_vocabulary1 = sorted(list(set(img_caption_vocabulary)))


In [10]:
remove_list = []
c = 0
for i in set(img_caption_vocabulary1):
    c = c+ 1
    if c %100 == 0:
        print(c)
    if img_caption_vocabulary.count(i) < 3:
         img_caption_vocabulary1.remove(i)
    

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600


In [11]:
print(len(img_caption_vocabulary))
print(len(img_caption_vocabulary1))


137616
5716


In [12]:

pickle.dump(img_caption_vocabulary1, open(DATA+"_image_caption_vocabulary_final.p", "wb"))  # save it into a file named save.p


In [13]:
# feature vector = title + bib_title+bib_venue

def get_feature_vector(x,img_caption,prefix_value,feature_vector):

    for file in x:
        prefix = prefix_value+file[:len(file)-5]+".pdf.json"
        feature_vector[prefix] = bagofwords(img_caption[prefix],img_caption_vocabulary1)


In [14]:
feature_vector = {}
x = listdir(img_text_path_1)
get_feature_vector(x,image_caption,'pos_',feature_vector)
x = listdir(img_text_path_2)
get_feature_vector(x,image_caption,'neg_',feature_vector)



In [15]:

pickle.dump(feature_vector, open("JNCA_train_image_caption_feature_vector.p", "wb"))  # save it into a file named save.p


In [16]:
img_text_path_1 = DATA+"/test/positive_image_caption"
img_text_path_2 = DATA+"/test/negative_image_caption"

In [17]:
feature_vector = {}
#for  jnca get unigrams or each element is dictionary of prefix_filename and corresponding unigram words preprocessed
x = listdir(img_text_path_1)
prefix = 'pos_'
image_caption = {}
get_unigrams(x,img_text_path_1,image_caption,prefix)
#for comnet get unigrams or each element is dictionary of prefix_filename and corresponding unigram words preprocessed
x = listdir(img_text_path_2)
prefix = 'neg_'
get_unigrams(x,img_text_path_2,image_caption,prefix)


x = listdir(img_text_path_1)
get_feature_vector(x,image_caption,'pos_',feature_vector)
x = listdir(img_text_path_2)
get_feature_vector(x,image_caption,'neg_',feature_vector)

pickle.dump(feature_vector, open(DATA+"_test_image_caption_feature_vector.p", "wb"))  # save it into a file named save.p
